In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

<ipython-input-4-b11c832c5082>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = pd.DatetimeIndex(train['일자']).week
<ipython-input-4-b11c832c5082>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = pd.DatetimeIndex(test['일자']).week


In [5]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [6]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [7]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

## 공휴일 변수 생성

In [8]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [9]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수',]]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수']]

In [10]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수'],
      dtype='object')

In [11]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 15)
(50, 15)


In [12]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 15)
(50, 15)


In [13]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


In [14]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

In [15]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수'],
      dtype='object')

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [19]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error as MAE
from IPython.display import clear_output
import sklearn


In [24]:
lunch_param = {'reg_alpha': 1.622564509910422e-05, 'reg_lambda': 0.0727906280606794, 'max_depth': 12, 'num_leaves': 7, 'colsample_bytree': 0.7041252632987905, 'subsample': 0.9930902085857553, 'subsample_freq': 5, 'min_child_samples': 24, 'max_bin': 437}
dinner_param = {'reg_alpha': 1.718625207505066e-06, 'reg_lambda': 0.03536226882919215, 'max_depth': 13, 'num_leaves': 149, 'colsample_bytree': 0.7238257801656021, 'subsample': 0.9792806922458114, 'subsample_freq': 3, 'min_child_samples': 26, 'max_bin': 423}

In [21]:
drop_columns = [[],['식사가능자수'], ['본사휴가자수'], ['본사출장자수'], ['인원변화']]
ks = [2,4,6,8,10]

In [22]:
random_seed = 0
pred_lunch = np.array([0]*50).astype(np.float64)

for dc in drop_columns:
    for k in ks:
        print(dc)
        
        lunch_model = LightGBMRegressor(random_state=random_seed)
        
        X_train = lunch_train.drop(columns=dc)
        X_test = lunch_test.drop(columns=dc)
        
        lunch_model.opt(X_train, y_lunch,
                        verbose=3,
                        cold_start=120,
                        folds=k,
                        opt_lvl=3,
                        early_stoping=120,
                        auto_parameters=False,
                        timeout=1100,
                        metric=MAE
                       )

        pred_lunch += lunch_model.predict(X_test)

        clear_output(True)
                
        random_seed += 1

23:09:06 | regression optimize: minimize
23:09:06 | ##################################################


[]


23:09:06 | > Step 1: calc parameters and pruned score: get test 10 trials


KeyboardInterrupt: 

In [42]:
random_seed = 0
pred_dinner = np.array([0]*50).astype(np.float64)

for dc in drop_columns:
    for k in ks:
        print(dc)
        
        dinner_model = LightGBMRegressor(random_state=random_seed)
        
        X_train = dinner_train.drop(columns=dc)
        X_test = dinner_test.drop(columns=dc)
        
        dinner_model.opt(X_train, y_dinner,
                        verbose=3,
                        cold_start=120,
                        folds=k,
                        opt_lvl=3,
                        early_stoping=120,
                        auto_parameters=False,
                        timeout=1100,
                        metric=MAE
                       )
            
        print(dc,random_seed,k)
            
        pred_dinner += dinner_model.predict(X_test)

        clear_output(True)
                
        random_seed += 1

17:04:36 | regression optimize: minimize
17:04:36 | ##################################################


['인원변화']


17:04:37 | > Step 1: calc parameters and pruned score: get test 10 trials
17:04:43 |  One iteration ~ 0.6 sec
17:04:43 |  Possible iters ~ 1754.0
17:04:43 | --------------------------------------------------
17:04:43 |   Pruned Threshold Score: 66.5278
17:04:43 | ##################################################
17:04:43 | > Step 2: Full opt with Threshold Score Pruner
17:04:43 | ##################################################
17:04:43 | > Start optimization with the parameters:
17:04:43 | CV_Folds = 10
17:04:43 | Score_CV_Folds = 2
17:04:43 | Feature_Selection = False
17:04:43 | Opt_lvl = 3
17:04:43 | Cold_start = 120
17:04:43 | Early_stoping = 120
17:04:43 | Metric = mean_absolute_error
17:04:43 | Direction = minimize
17:04:43 | ##################################################
Optimize: : 118it [01:50,  1.07it/s, | Model: LightGBM | OptScore: 58.7224 | Best mean_absolute_error: 57.3706 ]
17:06:33 | 
 EarlyStopping Exceeded: Best Score: 57.3706 mean_absolute_error
17:06:33 | > F

Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations


17:06:35 | Best Score: 57.3706 mean_absolute_error


Finished loading model, total used 400 iterations
Finished loading model, total used 400 iterations
['인원변화'] 24 10


In [45]:
submission = pd.read_csv('../data/sample_submission.csv')
N = len(ks)*len(drop_columns)

submission.iloc[:,1] = pred_lunch/N
submission.iloc[:,2] = pred_dinner/N
submission.head()

,일자,중식계,석식계
0,2021-01-27,991.486653,329.091166
1,2021-01-28,920.097761,419.711235
2,2021-01-29,629.656172,221.335990
3,2021-02-01,1303.890239,547.042724
4,2021-02-02,1056.245561,473.475649


In [48]:
answer = pd.read_csv('../submission/20210630_lgbm_autoML.csv')

lunch_answer = np.array(answer.iloc[:,1])
dinner_answer = np.array(answer.iloc[:,2])

abs(pred_lunch/N - lunch_answer).mean(), abs(pred_dinner/N - dinner_answer).mean()

(16.49537088386855, 9.859359321929544)

# 저장

In [49]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_lgbm_knfold_drop.csv', index =False)

오늘 날짜 : 20210701
